# Accessing ImageJ from Python Notebooks

**Part of the IAFIG-RMS *Python for Bioimage Analysis* Course.**

*Dr Chas Nelson* (based loosely on https://github.com/imagej/tutorials)

2019-12-11 1100--1230

## Aim

To introduce the `pyimagej` module and complete a basic pipeline using ImageJ functions.

## ILOs

* Be able to use `pyimagej` to access an ImageJ instance and the ImageJ Server API
* Be able to load and retreive data from ImageJ [Server] instances
* Be able to run ImageJ functions, scripts and plugins on image objects
* Appreciate that GPU-accelaration is possible through the use of `clijpy`
* Understand that python-ImageJ interfaces are all at an experimental stage but that they could be powerful additions to a pipeline.

## Data

The image we will use for the rest of this tutorial is from the Broad Bioimage Benchmark Collection data set BBBC0034v1 (https://data.broadinstitute.org/bbbc/; Thirstrup et al. 2018).

See https://data.broadinstitute.org/bbbc/BBBC034/ for the full description; however, the key points are:

* $1024 \times 1024 \times 52$ pixels
* $65 \times 65 \times 290$ nm/pixel
* 4 channels (each stored as separate files):
  * Cell membrane label (C=0)
  * Actin label (C=1)
  * DNA label (C=2)
  * Brightfield image (C=3)
  
The below cell can be run to create a local link to the data that we downloaded on the first day of the course. You only need to run this cell once and then you may comment it out.

In [ ]:
import os
from skimage import io

os.mkdirs('./assets',exist_ok=True)  # make folder (even if exists)
os.symlink('../../../00_image-processing/01_images-in-python/assets/bbbc034v1/','./assets/bbbc034v1')

# Create a file with a single slice (we will use this later)
np_channel1 = io.imread('./assets/bbbc034v1/AICS_12_134_C=1.tif')
io.imsave('./assets/bbbc034v1/AICS_12_134_C=1_s=26.tif',np_channel1[26,:,:])

## Combining ImageJ and Python

Many of us are already well versed on how to process images with ImageJ, so why relearn all of these tasks in Python? Python implementations might use different parameters or different underlying algorithsm (in fact, this is even true between packages). However, fear not, `pyimagej` provides a way of interacting with ImageJ and FIJI plug-ins from within a Python environment.

A further advantage of this approach is the ability to combine ImageJ and FIJI algorithms with other Python tools, such as NumPy, SciPy, scikit-image and more.

## Starting ImageJ

* Unlike other Python modules, which provide a set of functions, `pyimagej` provides an object - an ImageJ instance.
* This instance can be passed in data, e.g. image; can run internal functions (through a method-like format); and has additional options to output data.
* Note: for this notebook we will use a local copy of FIJI (2.0.0-rc-69/1.52p) that has the Server update URL enabled.

In [ ]:
import scyjava_config
import imagej

# Set Java Maximum Heap Memory to 12 GB
scyjava_config.add_options('-Xmx12g')

# Initialiase a specific version of ImageJ with FIJI plug-ins
# This is reproducible for all notebook users (but doesn't have everything)
# ij = imagej.init('sc.fiji:fiji:2.0.0-pre-10', headless=False)

# Initialiase a local version of ImageJ with FIJI plug-ins
# This is reproducible for each user but not necesarily across computers (but has extras)
ij = imagej.init('/home/chas/Fiji.app', headless=False)

In [ ]:
print(ij.getVersion())  # get ImageJ version
print(ij.getApp().getInfo(True))  # get ImageJ version, Java version and heap memory info

## Converting to/from NumPy to Java

* So far we've dealt with images stored as NumPy arrays; however, ImageJ is a Java-based environment and we need to convert between Python/NumPy data types and Java/ImageJ equivalents.
* We do this by creating a Java object that 'points' to the underlying NumPy array using `ij.py_to_java()`.
  * This means that changing the Java object alters the underlying Numpy data too (in place).
* Converted NumPy arrays are of Java type `RandomAccessibleInterval` and can also work in place of `IterableInterval` types.

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Scroll through the output of the previous code cell, i.e. <code>ij.op().help('multiply')</code>. Can you find an available operation suitable for NumPy arrays that have been wrapped as Java objects?</div>

In [ ]:
import numpy as np

# Create two NumPy arrays with data
arr1 = np.array([[1, 2], [3, 4]])
print('Array 1:')
display(arr1)
arr2 = np.array([[5, 6], [7, 8]])
print('Array 2:')
display(arr2)

# Convert to ImageJ objects
j_arr1 = ij.py.to_java(arr1)
j_arr2 = ij.py.to_java(arr2)

# Create an empty NumPy array (for our outputs)
# Takes one parameter (here arr1) for shape and dtype information
# Input can be a NumPy array or an ImageJ object
arr_output = ij.py.new_numpy_image(arr1)  
# Convert to ImageJ object
j_arr_output = ij.py.to_java(arr_output)

# Multiply our two images together
ij.op().run('multiply', j_arr_output, j_arr1, j_arr2)

# Display
print('Array 1x2:')
display(arr_output)

* We can convert Java objects into NumPy arrays using `ij.py.from_java()`.
  * Note that our NumPy axis convention is to use $z$, $x$, $y$, $\lambda$ and ImageJ uses $\lambda$, $x$, $y$, $z$.
  * `pyimagej` deals automatically with 3D images/arrays but for colours we need to manually correct axes and dimensions.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

# Open a 3D greyscale image directly in ImageJ (from a previous notebook)
ij_channel1 = ij.io().open('./assets/bbbc034v1/AICS_12_134_C=1.tif')

# Convert it to a NumPy array
np_channel1 = ij.py.from_java(ij_channel1)

# Print types and shapes
print('ImageJ object is of type: {0};\tNumPy object is of type: {1}.'.format(type(ij_channel1),type(np_channel1)))
# Note how the ImageJ function can work on both the ij and np objects
print('ImageJ object is of shape: {0};\tNumPy object is of shape: {1}.'.format(ij.py.dims(ij_channel1),ij.py.dims(np_channel1)))

# Display the a slice of the image
f, ax = plt.subplots(1,1)  # Create one subplot (1x1 grid)

ax.grid(False)
ax.imshow(np_channel1[26,:,:], cmap='gray', interpolation='none')

plt.show()

In [ ]:
from skimage import exposure

# Open a 3D multichannel image directly in ImageJ (from a previous notebook)
ij_allChannels = ij.io().open('./assets/bbbc034v1/AICS_12_134_C=all.tif')

# Convert it to a multi-dimensional NumPy array
np_allChannels = ij.py.from_java(ij_allChannels)

# Print types and shapes
print('ImageJ object is of type: {0};\tNumPy object is of type: {1}.'.format(type(ij_allChannels),type(np_allChannels)))
# Note how the ImageJ function can work on both the ij and np objects
print('ImageJ object is of shape: {0};\tNumPy object is of shape: {1}.'.format(ij.py.dims(ij_allChannels),ij.py.dims(np_allChannels)))

# Correct axis ordering
np_allChannels = np.moveaxis(np_allChannels,1,-1)
print('Axes Reordered: NumPy object is of shape: {0}.'.format(ij.py.dims(np_allChannels)))

# Rescale each channel independently for visualisation purposes
for channel in np.arange(np_allChannels.shape[-1]):
    np_allChannels[:,:,:,channel] = exposure.rescale_intensity(np_allChannels[:,:,:,channel],in_range='image',out_range='float64')

# Display the a slice of the image mapping the first three channels to RGB
f, ax = plt.subplots(1,1)  # Create one subplot (1x1 grid)

ax.grid(False)
ax.imshow(np_allChannels[26,:,:,:3], interpolation='none')

plt.show()

## ImageJ Ops

* `pyimagej` provides access to everything inside the ImageJ Ops framework
* We can get a list of these operations using `pyimagej`'s helpful `help()` function

In [ ]:
# List all Op 'methods', many of which are modules
print(ij.op().help())

# List all erosion methods and parameters
print(ij.op().help('erode'))

In [ ]:
dog_results = ij.py.new_numpy_image(np_allChannels[26,:,:,0])

sigma1 = 20
sigma2 = 4
ij.op().filter().dog(ij.py.to_java(dog_results), ij.py.to_java(np_allChannels[26,:,:,0]), sigma1, sigma2)

# Display
f, axes = plt.subplots(1,2)  # Create one subplot (1x2 grid)
(aO, aD) = axes.flatten()

aO.grid(False)
aO.imshow(np_allChannels[26,:,:,0], cmap='gray', interpolation='none')

aD.grid(False)
aD.imshow(dog_results, cmap='gray', interpolation='none')

plt.show()

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Create a new cell below and, using ImageJ Ops, run Otsu thresholding on the central slice from the DNA channel (channel 2) of our data. Then run morphological closing to tidy the data. Display the original, segmented and closed images.</div>

## Running ImageJ Scripts and Plug-ins

### Scripts

* Often people build up macros and scripts with time, rather than having to rewrite them all for Python, `pyimagej` provides access to the ImageJ server APIs ability to run scripts with `ij.py.run_script`.
* Scripts can be in any ImageJ compatible language, e.g. the ImageJ Macro language or Groovy.

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Modify the cell below with your own name and run the cell.</div>

In [ ]:
language = 'groovy'  # the script language file extension (in this case groovy)
script = """
#@String name
#@output String hello
hello = "Hello world! Hello " + name + "!"
"""  # The script, wrapped in triple quotes
args = {'name': 'Chas'}  # The input parameters as a dictionary
result = ij.py.run_script(language, script, args)
print(result.getOutput('hello'))  # get the output object

### Plug-ins

* Plugins work the same as scripts.
* You essentially run the plugin by name and pass the arguments in a dict.

In [ ]:
# Load UI and send NumPy Array
ij.ui().showUI()
np_Slice = np_allChannels[26,:,:,0]
ij.ui().show('Image',ij.py.to_java(np_Slice))

# Run Plugin
plugin = 'Mean'
args = { 
    'block_radius_x': 100,
    'block_radius_y': 100
}
ij.py.run_plugin(plugin, args)

# Create a window manager for interacting with the UI
windowManager = jnius.autoclass('ij.WindowManager')

# Get back data
np_SliceBlurred = ij.py.from_java(windowManager.getCurrentImage())
display(np.sum(np_Slice-np_SliceBlurred))

# Display
f, axes = plt.subplots(1,2)  # Create one subplot (1x1 grid)
(aO, aM) = axes.flatten()

aO.grid(False)
aO.imshow(np_allChannels[26,:,:,0], cmap='gray', interpolation='none')

aM.grid(False)
aM.imshow(np_SliceBlurred, cmap='gray', interpolation='none')

plt.show()

# Clear ImageJ windows
windowManager.closeAllWindows()

<div style="background-color:#abd9e9; border-radius: 5px; padding: 10pt"><strong>Task:</strong> Create a new cell below and run a 3D mean filter on the whole stack using an ImageJ instance. How might you work out the parameter names? Display the image with a widget so you can scroll through slices.</div>

## Accessing the ImageJ Server API

* So far we've used `pyimagej` to access a running instance of ImageJ; however, there is also an in-development system called the ImageJ Server.
* The ImageJ Server is an attempt to make a universal API thus making ImageJ for easy to interact with from non-Java language, i.e. Python.
* `pyimagej` wraps this API into simple Python function, something we can exploit.
* Before we can access the API, we must start the server.
  * Go to the open ImageJ UI and follow Plugins>Utilities>Start Server.

In [ ]:
# We need to manually get the imagej.server file as it currently isn't included in the module
# Comment this out once downloaded
import urllib.request
url = 'https://raw.githubusercontent.com/imagej/pyimagej/master/imagej/server/server.py'
urllib.request.urlretrieve(url,'./server.py')

import server as ijserver
import json

# Start the Server (either graphically or using the following)
ij.py.run_script('ijm', 'run("Start Server")')

# Initialise server API instance
ijs = ijserver.IJ()

# Search for a function with a vague term
options_create = ijs.find("create")
print('Options for the search "create":')
display(options_create)

# Get details of a function with a specific name
# Note the inputs, their types and requirements
# As well as the outputs
createImgFromImgID = ijs.find("CreateImgFromImg")[0]
details_createImgFromImg = ijs.detail(createImgFromImgID)
print('Details for the function "CreateImgFromImg":')
print(json.dumps(details_createImgFromImg, indent=4))

# Search for an invert function
options_invert = ijs.find("invert")
print('Options for the search "invert":')
display(options_invert)

invertIIID = ijs.find("InvertII")[0]
details_invertII = ijs.detail(invertIIID)
print('Details for the function "InvertII":')
print(json.dumps(details_invertII, indent=4))

# # Upload an image (by filename)
inputImage = ijs.upload('./assets/bbbc034v1/AICS_12_134_C=1_s=26.tif')

# Execute modules
result_createImgFromImg = ijs.run(createImgFromImgID, {'in': inputImage})
np_createImgFromImg = result_createImgFromImg['out']
result_invertII = ijs.run(invertIIID, {'in': inputImage, 'out': np_createImgFromImg})
np_invertII = result_invertII['out']

# Retrieve images
test = ijs.retrieve(np_invertII, format='Tiff', dest='./assets/bbbc034v1/AICS_12_134_C=1_s=26_inverted.tif')

# Close IJ windows
closeAllID = ijs.find("close-all")[0]
ijs.run(closeAllID)

# Load into Python
np_channel1 = io.imread('./assets/bbbc034v1/AICS_12_134_C=1_s=26.tif')
np_channel1_inverted = io.imread('./assets/bbbc034v1/AICS_12_134_C=1_s=26_inverted.tif')
print(np_channel1.shape,np_channel1_inverted.shape)

# Display
f, axes = plt.subplots(1,2)  # Create one subplot (1x1 grid)
(aO, aI) = axes.flatten()

aO.grid(False)
aO.imshow(np_channel1, cmap='gray', interpolation='none')
aO.set_title('Original')

aI.grid(False)
aI.imshow(np_channel1_inverted, cmap='gray', interpolation='none')
aI.set_title('Inverted')

plt.show()

## CLIJPY - Utilising the GPU

From https://clij.github.io:
>CLIJ is an OpenCL - ImageJ bridge and a Fiji plugin allowing users with entry-level skills in programming to build GPU-accelerated workflows to speed up their image processing.

* `CLIJPY` is a python wrapper for `CLIJ` using `pyimagej`.
* See https://github.com/clij/clijpy/blob/master/python/clijpy_demo.ipynb for a demo using `clijpy` to threshold and label an image of blobs.

## Summary

* `pyimagej` provides Python-level access an ImageJ instance and the ImageJ Server API
* Data can be passed to an ImageJ instance and retrieved from an instance but must be converted between Python and Java types.
* ImageJ functions/scripts/plugins can be run through Python - but not all work out of the box.
* GPU-accelaration is possible through the use of `clijpy` (not covered in this material).
* Python-ImageJ interfaces are all at an experimental stage but that they could be powerful additions to a pipeline.